In [21]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [22]:
raw = pd.read_csv("/content/drive/MyDrive/Barley_baseline.csv")

In [23]:
car = raw['CARUID'].unique()
missing_car = pd.DataFrame(columns = ["CAR","# of missing year"])
available_years = [1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,
                   1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,
                   2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

In [24]:
for c in car:

  ##Select with rows of specific CARUID
  sub = raw.loc[raw['CARUID'] == c]

  ##Dataframe with all -999 values removed
  df = sub[(sub.iloc[:, 1:] != -999).all(axis=1)]

  ##Get all unique, available years
  yr = df['Year'].unique()
  in_var = ["SumPcpn", "SumEGDD_C", "SumHeatD", "SumFrostD", "AvgSI", "AvgPrcnAWHC", "NDVI"]

  ##Set up empty list to make it easier to select column names later
  SumPcpn_avg = []
  SumEGDD_C_avg = []
  SumHeatD_avg = []
  SumFrostD_avg = []
  AvgSI_avg = []
  AvgPrcnAWHC_avg = []
  NDVI = []

  ##Calculates 3 week moving average for each input variable and puts that column
  ##into original dataframe.
  for i in list(range(18,37)):
    df['SumPcpn' + str(i) + "_" + str(i + 2)] = df[['SumPcpn' + str(i), 'SumPcpn' + str(i + 1),
                                                    'SumPcpn' + str(i + 2)]].mean(axis=1)
    ##Add column name to the input variable list
    SumPcpn_avg.append('SumPcpn' + str(i) + "_" + str(i + 2))

    df['SumEGDD_C' + str(i) + "_" + str(i + 2)] = df[['SumEGDD_C' + str(i), 'SumEGDD_C' + str(i + 1),
                                                    'SumEGDD_C' + str(i + 2)]].mean(axis=1)
    SumEGDD_C_avg.append('SumEGDD_C' + str(i) + "_" + str(i + 2))

    df['SumHeatD' + str(i) + "_" + str(i + 2)] = df[['SumHeatD' + str(i), 'SumHeatD' + str(i + 1),
                                                    'SumHeatD' + str(i + 2)]].mean(axis=1)
    SumHeatD_avg.append('SumHeatD' + str(i) + "_" + str(i + 2))

    df['SumFrostD' + str(i) + "_" + str(i + 2)] = df[['SumFrostD' + str(i), 'SumFrostD' + str(i + 1),
                                                    'SumFrostD' + str(i + 2)]].mean(axis=1)
    SumFrostD_avg.append('SumFrostD' + str(i) + "_" + str(i + 2))

    df['AvgSI' + str(i) + "_" + str(i + 2)] = df[['AvgSI' + str(i), 'AvgSI' + str(i + 1),
                                                    'AvgSI' + str(i + 2)]].mean(axis=1)
    AvgSI_avg.append('AvgSI' + str(i) + "_" + str(i + 2))

    df['AvgPrcnAWHC' + str(i) + "_" + str(i + 2)] = df[['AvgPrcnAWHC' + str(i), 'AvgPrcnAWHC' + str(i + 1),
                                                    'AvgPrcnAWHC' + str(i + 2)]].mean(axis=1)
    AvgPrcnAWHC_avg.append('AvgPrcnAWHC' + str(i) + "_" + str(i + 2)) 
    NDVI.append('NDVI' + str(i) + "_" + str(i + 2))

  ##The final dataframe that will be used as final output
  column_names = ["Year", "SumPcpn", "SumEGDD_C", "SumHeatD","SumFrostD", "AvgSI","AvgPrcnAWHC", "NDVI"]
  fin_table = pd.DataFrame(columns = column_names)

  ##Loop through all available years
  for y in yr:

    yr_window = []

    for j in list(range(18,37)):
      yr_window.append(str(y) + "_" + str(j) + "_" + str(j + 2))

    ##Get all the input variables of the specific year (y) that has been 3 week averaged
    ##This selects specifically selects columns that has the input variable in its name
    SumPcpn_transp = df[SumPcpn_avg].loc[df["Year"] == y].values.tolist()[0]
    SumEGDD_C_transp = df[SumEGDD_C_avg].loc[df["Year"] == y].values.tolist()[0]
    SumHeatD_transp = df[SumHeatD_avg].loc[df["Year"] == y].values.tolist()[0]
    SumFrostD_transp = df[SumFrostD_avg].loc[df["Year"] == y].values.tolist()[0]
    AvgSI_transp = df[AvgSI_avg].loc[df["Year"] == y].values.tolist()[0]
    AvgPrcnAWHC_transp = df[AvgPrcnAWHC_avg].loc[df["Year"] == y].values.tolist()[0]
    NDVI_transp = df[NDVI].loc[df["Year"] == y].values.tolist()[0]

    ##Make temporary dataframe and put the 3-week average input variable there
    inter = pd.DataFrame(data={"Year": yr_window})
    inter["SumPcpn"] = SumPcpn_transp
    inter["SumEGDD_C"] = SumEGDD_C_transp
    inter["SumHeatD"] = SumHeatD_transp
    inter["SumFrostD"] = SumFrostD_transp
    inter["AvgSI"] = AvgSI_transp
    inter["AvgPrcnAWHC"] = AvgPrcnAWHC_transp
    inter["NDVI"] = NDVI_transp

    ##Append by row to fin_table
    fin_table = fin_table.append(inter, ignore_index=True)
    missing_amount = len(available_years) - len(yr)

    
  missing_car = missing_car.append(pd.DataFrame(data={"CAR": c, "# of missing year": missing_amount}, index=[0]),
                                     ignore_index=False)
  fin_table.to_csv("/content/drive/MyDrive/Final_caruid_csv/" + "BARLEY_" + str(c) + ".csv", index=False)

missing_car.to_csv("/content/drive/MyDrive/Final_caruid_csv/" + "BARLEY_missing_data.csv", index=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab